## 📒 Notebook Exploratorio - Evaluación de Calidad de Datos PLANEA

In [1]:
# 📒 Notebook Exploratorio - Evaluación de Calidad de Datos PLANEA

# Paso 1: Cargar librerías necesarias
import pandas as pd
import unidecode
import numpy as np
import matplotlib.pyplot as plt

# Paso 2: Cargar archivo consolidado
df = pd.read_csv("../output/planea/planea_total.csv", encoding="utf-8-sig", low_memory=False)
print(f"✅ Archivo cargado con {df.shape[0]:,} filas y {df.shape[1]} columnas")

# Paso 3: Vista general de las variables
df.info()

# Paso 4: Vista previa del contenido
df.head(3)

✅ Archivo cargado con 321,513 filas y 76 columnas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321513 entries, 0 to 321512
Data columns (total 76 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ANIO_EVALUACION                       321513 non-null  int64  
 1   ENT                                   321513 non-null  object 
 2   ENTIDAD                               321513 non-null  object 
 3   CLAVE_ESCUELA                         321513 non-null  object 
 4   NOMBRE_ESCUELA                        321444 non-null  object 
 5   TURNO                                 321513 non-null  object 
 6   MUNICIPIO                             321513 non-null  object 
 7   LOCALIDAD                             321513 non-null  object 
 8   TIPO_ESCUELA                          321513 non-null  object 
 9   GRADO_EVALUADO                        321513 non-null  object 
 10  GRADO_MARGINACION 

,ANIO_EVALUACION,ENT,ENTIDAD,CLAVE_ESCUELA,NOMBRE_ESCUELA,TURNO,MUNICIPIO,LOCALIDAD,TIPO_ESCUELA,GRADO_EVALUADO,...,PERCEPCION_BIENESTAR_NACIONAL_PCT_2,PERCEPCION_BIENESTAR_NACIONAL_PCT_3,PERCEPCION_BIENESTAR_NACIONAL_PCT_4,MARGINACION_PARECIDAS_ALTO,MARGINACION_PARECIDAS_MEDIO,MARGINACION_PARECIDAS_BAJO,MARGINACION_NACIONAL_ALTO,MARGINACION_NACIONAL_MEDIO,MARGINACION_NACIONAL_BAJO,ARCHIVO_ORIGEN
0,2015,1,AGUASCALIENTES,01DPR0001V,RODRIGO RINCON GALLARDO,MATUTINO,RINCÓN DE ROMOS,RINCON DE ROMOS,General Pública,sexto año de primaria,...,33.3,39.3,11.6,0.0,100.0,0.0,39.2,30.3,29.0,2015_PEB_Escuelas_01.xlsx
1,2015,1,AGUASCALIENTES,01DPR0002U,ANTONIO VENTURA MEDINA,COMPLETO,SAN JOSÉ DE GRACIA,SAN JOSE DE GRACIA,General Pública,sexto año de primaria,...,33.3,39.3,11.6,38.4,32.6,27.7,39.2,30.3,29.0,2015_PEB_Escuelas_01.xlsx
2,2015,1,AGUASCALIENTES,01DPR0003T,BENIGNO CHAVEZ,VESPERTINO,JESÚS MARÍA,JESUS MARIA,General Pública,sexto año de primaria,...,33.3,39.3,11.6,100.0,0.0,0.0,39.2,30.3,29.0,2015_PEB_Escuelas_01.xlsx


### Revisión variable por variable

In [2]:
# Listado completo de columnas
for col in df.columns:
    print(f"\n🔍 Variable: {col}")
    print(df[col].value_counts(dropna=False).head(10))  # Las 10 categorías más frecuentes
    print(f"Tipo: {df[col].dtype} | Nulos: {df[col].isna().sum()} | Únicos: {df[col].nunique()}")



🔍 Variable: ANIO_EVALUACION
ANIO_EVALUACION
2015    105404
2016    105148
2018     76990
2017     33971
Name: count, dtype: int64
Tipo: int64 | Nulos: 0 | Únicos: 4

🔍 Variable: ENT
ENT
30    36701
15    33932
14    22038
21    19955
11    18031
12    14730
24    13733
9     13371
13    12499
19    10940
Name: count, dtype: int64
Tipo: object | Nulos: 0 | Únicos: 33

🔍 Variable: ENTIDAD
ENTIDAD
VERACRUZ            36701
JALISCO             22038
PUEBLA              19955
GUANAJUATO          18031
GUERRERO            14730
HIDALGO             12499
SAN LUIS POTOSI     12072
ESTADO DE MÉXICO    11372
ESTADO DE MEXICO    11147
NUEVO LEON           9867
Name: count, dtype: int64
Tipo: object | Nulos: 0 | Únicos: 41

🔍 Variable: CLAVE_ESCUELA
CLAVE_ESCUELA
09DST0067H    6
26EST0017P    6
14PES0129B    6
26ETV0218Z    6
30DST0116C    6
14DST0001K    6
30DES0012P    6
14DST0002J    6
14DST0003I    6
30DES0015M    6
Name: count, dtype: int64
Tipo: object | Nulos: 0 | Únicos: 119058

🔍 Variabl

### Limpieza: Variable `ENT`

In [3]:
# Copia original para trazabilidad
df["ENT_ORIGINAL"] = df["ENT"]

# Diagnóstico inicial
print("🔍 Valores únicos antes de limpiar:")
print(sorted(df["ENT"].unique()))

# Verifica si existen valores no convertibles a enteros
ent_invalidos = df[~df["ENT"].astype(str).str.match(r"^\s*\d+\s*$")]

if not ent_invalidos.empty:
    print("\n🚨 Registros con valores no enteros en 'ENT' antes de limpiar:")
    display(ent_invalidos[["ENT", "ANIO_EVALUACION", "ARCHIVO_ORIGEN"]].head(10))
    print(f"Total registros no enteros detectados: {len(ent_invalidos)}")

# Paso 1: Limpieza de caracteres no numéricos
df["ENT"] = (
    df["ENT"]
    .astype(str)
    .str.strip()
    .str.replace(r"[^0-9]", "", regex=True)
)

# Paso 2: Conversión segura a enteros
df["ENT"] = pd.to_numeric(df["ENT"], errors="coerce").astype("Int64")

# Paso 3: Diagnóstico final
print("\n✅ Valores únicos después de limpiar:")
print(sorted(df["ENT"].dropna().unique()))
print(f"Tipo: {df['ENT'].dtype} | Nulos: {df['ENT'].isna().sum()} | Únicos: {df['ENT'].nunique()}")

# Paso 4 (opcional): Mostrar registros con valores faltantes
if df["ENT"].isna().sum() > 0:
    print("\n⚠️ Registros con 'ENT' nulo tras limpieza:")
    display(df[df["ENT"].isna()][["ENT_ORIGINAL", "ANIO_EVALUACION", "ARCHIVO_ORIGEN"]].head(10))


🔍 Valores únicos antes de limpiar:
['1', '10', '11', '12', '13', '14', '15', '16', '16_', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '4', '5', '6', '7', '8', '9']

🚨 Registros con valores no enteros en 'ENT' antes de limpiar:


,ENT,ANIO_EVALUACION,ARCHIVO_ORIGEN
283595,16_,2018,2018_PEB_Escuelas_16.xlsx


Total registros no enteros detectados: 1

✅ Valores únicos después de limpiar:
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32)]
Tipo: Int64 | Nulos: 0 | Únicos: 32


In [4]:
print(sorted(df["ENTIDAD"].unique()))

['AGUASCALIENTES', 'BAJA CALIFORNIA', 'BAJA CALIFORNIA SUR', 'CAMPECHE', 'CHIAPAS', 'CHIHUAHUA', 'CIUDAD DE MEXICO', 'CIUDAD DE MÉXICO', 'COAHUILA', 'COLIMA', 'DURANGO', 'ESTADO DE MEXICO', 'ESTADO DE MÉXICO', 'GUANAJUATO', 'GUERRERO', 'HIDALGO', 'JALISCO', 'MEXICO', 'MICHOACAN', 'MICHOACÁN', 'MORELOS', 'MÉXICO', 'NAYARIT', 'NUEVO LEON', 'NUEVO LEÓN', 'OAXACA', 'PUEBLA', 'QUERETARO', 'QUERÉTARO', 'QUINTANA ROO', 'SAN LUIS POTOSI', 'SAN LUIS POTOSÍ', 'SINALOA', 'SONORA', 'TABASCO', 'TAMAULIPAS', 'TLAXCALA', 'VERACRUZ', 'YUCATAN', 'YUCATÁN', 'ZACATECAS']


### Limpieza: Variable `ENTIDAD`

In [5]:
from unidecode import unidecode

# Diccionario de claves ENT a nombre de entidad
ENTIDADES = {
    1: "AGUASCALIENTES", 2: "BAJA CALIFORNIA", 3: "BAJA CALIFORNIA SUR", 4: "CAMPECHE",
    5: "COAHUILA", 6: "COLIMA", 7: "CHIAPAS", 8: "CHIHUAHUA", 9: "CIUDAD DE MEXICO",
    10: "DURANGO", 11: "GUANAJUATO", 12: "GUERRERO", 13: "HIDALGO", 14: "JALISCO",
    15: "ESTADO DE MEXICO", 16: "MICHOACAN", 17: "MORELOS", 18: "NAYARIT", 19: "NUEVO LEON",
    20: "OAXACA", 21: "PUEBLA", 22: "QUERETARO", 23: "QUINTANA ROO", 24: "SAN LUIS POTOSI",
    25: "SINALOA", 26: "SONORA", 27: "TABASCO", 28: "TAMAULIPAS", 29: "TLAXCALA",
    30: "VERACRUZ", 31: "YUCATAN", 32: "ZACATECAS"
}

# 1. Convertimos a string y aplicamos limpieza general
df["ENTIDAD"] = df["ENTIDAD"].astype(str).str.strip()
df["ENTIDAD"] = df["ENTIDAD"].apply(lambda x: unidecode(x.upper()))

# 2. Reemplazo específico
df["ENTIDAD"] = df["ENTIDAD"].replace("MEXICO", "ESTADO DE MEXICO")

# 3. Lista válida (de referencia) de entidades esperadas
entidades_validas = set(ENTIDADES.values())

# 4. Identificamos valores no válidos antes de limpiar
valores_no_validos = set(df["ENTIDAD"].unique()) - entidades_validas
if valores_no_validos:
    print("⚠️ Valores no válidos encontrados en la columna 'ENTIDAD':")
    for val in sorted(valores_no_validos):
        print(f"  • {val}")
    print("\n🔍 Registros afectados:")
    print(df[df["ENTIDAD"].isin(valores_no_validos)][["ENTIDAD", "ARCHIVO_ORIGEN"]].head(10))  # muestra ejemplos

# 5. Opción: si quieres forzar que los no válidos se pongan como pd.NA
df.loc[~df["ENTIDAD"].isin(entidades_validas), "ENTIDAD"] = pd.NA

# 6. Finalmente, nos aseguramos de que el tipo sea string
df["ENTIDAD"] = df["ENTIDAD"].astype("string")

# Diagnóstico final
print("\n✅ Valores únicos en 'ENTIDAD':")
valores_ordenados = df["ENTIDAD"].value_counts(dropna=False).sort_index()
print(valores_ordenados)

# Número total de valores únicos (incluyendo NaN como categoría aparte)
num_unicos = df["ENTIDAD"].nunique(dropna=False)
print(f"\n🔢 Total de valores únicos en 'ENTIDAD' (incluyendo nulos): {num_unicos}")

print(f"Tipo: {df['ENTIDAD'].dtype} | Nulos: {df['ENTIDAD'].isna().sum()} | Únicos: {df['ENTIDAD'].nunique()}")




✅ Valores únicos en 'ENTIDAD':
ENTIDAD
AGUASCALIENTES          3104
BAJA CALIFORNIA         6646
BAJA CALIFORNIA SUR     1760
CAMPECHE                3185
CHIAPAS                 8185
CHIHUAHUA               8698
CIUDAD DE MEXICO       13371
COAHUILA                7047
COLIMA                  1896
DURANGO                 9243
ESTADO DE MEXICO       33932
GUANAJUATO             18031
GUERRERO               14730
HIDALGO                12499
JALISCO                22038
MICHOACAN               3585
MORELOS                 4817
NAYARIT                 4898
NUEVO LEON             10940
OAXACA                  2855
PUEBLA                 19955
QUERETARO               5763
QUINTANA ROO            3676
SAN LUIS POTOSI        13733
SINALOA                 9704
SONORA                  7442
TABASCO                 8192
TAMAULIPAS              9095
TLAXCALA                3374
VERACRUZ               36701
YUCATAN                 3983
ZACATECAS               8435
Name: count, dtype: Int64

🔢 Tot